In [1]:
from pycoalescence import Simulation, CoalescenceTree
import random
from paleo_occ_sq import *

Starting module load...
Module load complete


In [2]:
global_save_dir = "output"
output_dir = "output"
data_dir = "/run/media/sam/Media/Paleo/Data/"
fragment_dir = "/run/media/sam/Media/Paleo/Data/configs"
output = []
for pbs_index in range(1, 100, 10):
    params = get_sim_parameters(pbs_index, global_save_dir)
    seed = params["seed"]
    random.seed(seed)
    job_type = params["job_type"]
    interval = params["interval"]
    tetrapod_group = params["tetrapod_group"]
    sigma = params["sigma"]
    density_per_km = params["density_per_km"]
    save_dir = params["output_directory"]
    sample_size, random_times = get_sample_size(interval, tetrapod_group, density_per_km)
    simulation = Simulation(logging_level=30)
    simulation.set_simulation_parameters(
        seed=seed,
        job_type=job_type,
        output_directory=output_dir,
        min_speciation_rate=0.9999,
        sigma=sigma,
        tau=0.0,
        deme=density_per_km,
        sample_size=sample_size,
        max_time=60 * 60 * 21.5,
        dispersal_method="normal",
        uses_spatial_sampling=True,
    )
    sample_file, fine_file, fragment_file = get_sim_type(
        interval, tetrapod_group, data_dir, fragment_dir, "pristine", 
    )
    simulation.set_map_files(sample_file=sample_file, fine_file=fine_file, coarse_file="none")
    for t in random_times:
        simulation.add_sample_time(time=t)

    try:
        simulation.run()
    except IOError:
        pass
    ct = CoalescenceTree(simulation)
    output.append(["{}-{}".format(job_type, seed), interval, tetrapod_group, ct.get_total_number_individuals(), density_per_km, sample_size, len(random_times)])

In [3]:
import pandas as pd

In [4]:
pd.DataFrame(output)

,0,1,2,3,4,5,6
0,2-2,artinskian,amniote,1539,121.998706,0.968248,2
1,2-12,artinskian,amniote,572,34.424688,0.686281,10
2,2-22,artinskian,amniote,1107,30.381943,0.972000,8
3,3-7,artinskian,amphibian,946,26.429407,0.893891,10
4,3-17,artinskian,amphibian,860,40.402603,0.584740,10
5,4-2,asselian,amniote,178,121.998706,0.309839,1
6,4-12,asselian,amniote,70,34.424688,0.122005,9
7,4-22,asselian,amniote,95,30.381943,0.311040,4
8,5-7,asselian,amphibian,264,26.429407,0.204318,7
9,5-17,asselian,amphibian,216,40.402603,0.103954,9


In [5]:
t = CoalescenceTree("output/data_4_12.db")
t.get_total_number_individuals()

70

In [6]:
pbs_index = 61
params = get_sim_parameters(pbs_index, global_save_dir)
seed = params["seed"]
random.seed(seed)
job_type = params["job_type"]
interval = params["interval"]
tetrapod_group = params["tetrapod_group"]
sigma = params["sigma"]
density_per_km = params["density_per_km"]
save_dir = params["output_directory"]
sample_size, random_times = get_sample_size(interval, tetrapod_group, density_per_km)
simulation = Simulation(logging_level=10)
simulation.set_simulation_parameters(
    seed=seed,
    job_type=job_type,
    output_directory="output/test/",
    min_speciation_rate=0.999,
    sigma=sigma,
    tau=0.0,
    deme=density_per_km,
    sample_size=sample_size,
    max_time=60 * 60 * 21.5,
    dispersal_method="normal",
    uses_spatial_sampling=True,
)
sample_file, fine_file, fragment_file = get_sim_type(
    interval, tetrapod_group, data_dir, fragment_dir, "pristine", 
)
simulation.set_map_files(sample_file=sample_file, fine_file=fine_file, coarse_file="none")
for t in random_times:
    simulation.add_sample_time(time=t)

try:
    simulation.run()
except IOError:
    pass

Checking folder existance.../run/media/sam/Media/Paleo/Data/paleo_maps/asselian.tif exists.               
Checking folder existance...output/test/ exists.               
Checking folder existance.../run/media/sam/Media/Paleo/Data/paleo_maps/paleomask_occ_sq_asselian_amniote.tif exists.               
Checking folder existance...done.                                                                
Checking for unfinished simulations...done.
No files found containing unfinished simulations.
*************************************************
Setting up simulation...
Dispersal (tau, sigma): 0, 4.72417
Dispersal method: normal
Fine map
-file: /run/media/sam/Media/Paleo/Data/paleo_maps/asselian.tif
-dimensions: (32072, 15486)
-offset: (10605, 5836)
Coarse map
-file: none
-dimensions: (32072, 15486)
-offset: (0, 0)
-scale: 1
Sample grid
-file: /run/media/sam/Media/Paleo/Data/paleo_maps/paleomask_occ_sq_asselian_amniote.tif
-dimensions: (5856, 1657)
-optimised area: (5856, 1657)
-optimised off

In [7]:
ct = CoalescenceTree(simulation)
ct.get_total_number_individuals()

FileNotFoundError: File output/test/data_4_12.db does not exist.

In [11]:
from pycoalescence import Map
import numpy as np
sample_map = Map(sample_file)
sample_map.open()
data = sample_map.data * density_per_km * sample_size
data = np.floor(data)
np.sum(data)

8.0

In [27]:
# data_dir = "/run/media/sam/Media/Paleo/Data/"
interval_map_path = os.path.join(data_dir, "paleo_maps", "{}.tif".format(interval))
interval_map = Map(interval_map_path)
x, y = sample_map.get_x_y()
x_off, y_off = sample_map.calculate_offset(interval_map)
np.floor(np.sum(interval_map.get_subset(x_off, y_off, x, y, no_data_value=0)*sample_map.data) * density_per_km * sample_size) * 9

1062.0

In [11]:
121*0.61*3

221.43

In [31]:
def compress_times(coalescence_tree):
    if coalescence_tree.database is None:
        raise IOError("Coalescence tree has not been opened")
    species_list = [list(x) for x in coalescence_tree.get_species_list()]
    coalescence_tree._backup_species_list()
    for each in species_list:
        if each[6] == 1:
            each[12] = 0.0
    coalescence_tree.cursor.execute(
        "CREATE TABLE SPECIES_LIST (ID int PRIMARY KEY NOT NULL, unique_spec INT NOT NULL, xval INT NOT NULL, "
        "yval INT NOT NULL, xwrap INT NOT NULL, ywrap INT NOT NULL, tip INT NOT NULL, speciated INT NOT "
        "NULL, parent INT NOT NULL, existence INT NOT NULL, randnum REAL NOT NULL, gen_alive INT NOT "
        "NULL, gen_added REAL NOT NULL)"
    )
    coalescence_tree.cursor.executemany(
        "INSERT INTO SPECIES_LIST (ID,unique_spec,xval,yval,xwrap,ywrap,tip,speciated,parent,existence, "
        "randnum,gen_alive,gen_added) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)",
        species_list,
    )
    coalescence_tree.database.commit()

In [32]:
c.wipe_data()
compress_times(c)

In [33]:
c.set_speciation_parameters(speciation_rates=[0.1, 0.2])
c.apply()

In [34]:
for ref in c.get_community_references():
    params = c.get_community_parameters(ref)
    print("Species richness in {} at time {} with sr {} was {} with {} individuals".format(ref, 
                                                                      params["time"],
                                                                      params["speciation_rate"],
                                                                      c.get_species_richness(ref),
                                                                                          c.get_number_individuals()))

Species richness in 1 at time 0.0 with sr 0.1 was 50 with 200 individuals
Species richness in 2 at time 0.0 with sr 0.2 was 85 with 200 individuals


In [2]:
import pandas as pd
from tqdm import tqdm_notebook
import os
import lhsmdu
import numpy as np
from collections import OrderedDict
from pycoalescence import CoalescenceTree

In [5]:
job_types = [10, 26, 42]
input_dir_fragmented = "/run/media/sam/Media/Paleo/Results/PaleoMain/Sim8/"
totals = [0, 0, 0]
for i, pc in enumerate([20, 40, 80]):
    folder = os.path.join(input_dir_fragmented, str(pc))
    for file in os.listdir(folder):
        try:
            ct = CoalescenceTree(os.path.join(folder, file))
        except Exception:
            continue
        params = ct.get_simulation_parameters()
        if job_types[i] == params["job_type"]:
            if "kasimovian" not in params["sample_file"] and "amniote" not in params["sample_file"]:
                raise ValueError("File at {} is not kasimovian, amniote".format(file))
            print(file)
            totals[i] += 1

data_10_1.db
data_10_10.db
data_10_12.db
data_10_11.db
data_10_13.db
data_10_14.db
data_10_15.db
data_10_16.db
data_10_17.db
data_10_18.db
data_10_19.db
data_10_2.db
data_10_20.db
data_10_21.db
data_10_22.db
data_10_23.db
data_10_3.db
data_10_24.db
data_10_4.db
data_10_5.db
data_10_6.db
data_10_7.db
data_10_8.db
data_10_9.db
data_26_10.db
data_26_1.db
data_26_11.db
data_26_12.db
data_26_13.db
data_26_14.db
data_26_15.db
data_26_16.db
data_26_17.db
data_26_18.db
data_26_19.db
data_26_2.db
data_26_20.db
data_26_21.db
data_26_22.db
data_26_23.db
data_26_24.db
data_26_3.db
data_26_4.db
data_26_5.db
data_26_6.db
data_26_7.db
data_26_8.db
data_26_9.db
data_42_1.db
data_42_10.db
data_42_11.db
data_42_12.db
data_42_14.db
data_42_15.db
data_42_13.db
data_42_16.db
data_42_17.db
data_42_18.db
data_42_19.db
data_42_2.db
data_42_20.db
data_42_21.db
data_42_22.db
data_42_23.db
data_42_24.db
data_42_3.db
data_42_4.db
data_42_5.db
data_42_6.db
data_42_7.db
data_42_8.db
data_42_9.db


In [6]:
totals

[24, 24, 24]

In [3]:
from pycoalescence import Map
sample_map = "/run/media/sam/Media/Paleo/Data/paleo_maps/paleomask_kasimovian_amniote.tif"
m = Map(sample_map)
m.open()
np.sum(m.data)

6.071428760886192

In [4]:
from pycoalescence import Simulation
density_per_km=200
sample_size = min(10*7 / density_per_km, 1.0)
simulation = Simulation(logging_level=10)
simulation.set_simulation_parameters(seed=1, job_type=1, output_directory="output",
                                     min_speciation_rate=10 ** -3, sigma=1.6, tau=0.0, deme=density_per_km,
                                     sample_size=sample_size, max_time=60 * 60 * 22.5, dispersal_method="normal",
                                     uses_spatial_sampling=True)
sample_file = "/run/media/sam/Media/Paleo/Data/paleo_maps/paleomask_kasimovian_amniote.tif"
fine_file = "/run/media/sam/Media/Paleo/Data/paleo_maps/fragmented/kasimovian_0.2_fragmented.tif"
simulation.set_map_files(sample_file=sample_file, fine_file=fine_file, coarse_file="none")
simulation.run()

Checking folder existance.../run/media/sam/Media/Paleo/Data/paleo_maps/fragmented/kasimovian_0.2_fragmented.tif exists.               
Checking folder existance...output exists.               
Checking folder existance.../run/media/sam/Media/Paleo/Data/paleo_maps/paleomask_kasimovian_amniote.tif exists.               
Checking folder existance...done.                                                                
Checking for unfinished simulations...done.
No files found containing unfinished simulations.
*************************************************
Setting up simulation...
Dispersal (tau, sigma): 0, 1.6
Dispersal method: normal
Fine map
-file: /run/media/sam/Media/Paleo/Data/paleo_maps/fragmented/kasimovian_0.2_fragmented.tif
-dimensions: (32072, 15486)
-offset: (11408, 6679)
Coarse map
-file: none
-dimensions: (32072, 15486)
-offset: (0, 0)
-scale: 1
Sample grid
-file: /run/media/sam/Media/Paleo/Data/paleo_maps/paleomask_kasimovian_amniote.tif
-dimensions: (4956, 907)
-optimise